# **Tech Companies Layoff Analysis**
## **by Mateus Lima Silva**

---

## 1. Importing required libraries

In [2]:
# Importing libraries
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt
import seaborn              as sns

## 2. Importing the dataset

In [3]:
# Default variables
df = pd.read_csv('../data/raw_tech_layoffs.csv', sep=',')

## 3. Exploring the dataset

In [6]:
# Getting 5 random rows to analyze the dataframe
df.sample(5)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes
172,Cedar,Unclear,24,7/7/2022,Health Care,New York,Business Insider,Private,NaN
21,CoinDCX,100,6,1/11/2023,"Blockchain, cryptocurrency","Mumbai, India",Biometric Update,Private,NaN
393,nCino,100,7,1/18/2023,"Fintech, software","Wilmington, NC",Port City Daily,Public,NaN
343,Sama,Unclear,Unclear,9/12/2022,"AI, enterprise software",San Francisco,Sama,Private,NaN
113,Citrix Systems,Unclear,15,1/10/2023,"Security, SaaS","Fort Lauderdale, FL",TechCrunch,Private,NaN


In [20]:
# Dataframe size
total_rows = df.shape[0]
total_columns = df.shape[1]
print('This dataframe has {} rows and {} columns.'.format(total_rows, total_columns))

This dataframe has 489 rows and 9 columns.


In [15]:
# Checking the available columns and their types
df.dtypes

company                          object
total_layoffs                    object
impacted_workforce_percentage    object
reported_date                    object
industry                         object
headquarter_location             object
sources                          object
status                           object
additional_notes                 object
dtype: object

In [23]:
# Counting unique companies in the dataset
unique_companies = df[['company']].nunique()[0]
unique_companies

477

In [95]:
# 489 - 477 = 12 repeated rows = 2 or more layoffs from the same company registered
# Checking which companies repeat in the dataset
repeated_companies = df[df['company'].duplicated() == True][['company', 'industry']]
repeated_companies

,company,industry
69,Gemini,"Fintech, Crypto"
92,Thirty Madison,"health care, wellness"
185,Homeward,Proptech
188,DataRobot,"AI, enterprise software"
201,Socure,Identity verification
211,TruePill,"pharmaceutical, health care"
249,Argo AI,Transportation
300,On Deck,"Networking, business development"
342,Sundae,PropTech
451,Blend,"Fintech, proptech"


In [98]:
# Adding the "reported_date" for the repeated_companies dataframe
repeated_companies = df[['company', 'reported_date']].groupby(['company'])['reported_date'].apply(', '.join).reset_index()
repeated_companies = repeated_companies.merge(df[['company', 'industry']], on='company', how='left')
repeated_companies = repeated_companies[repeated_companies['reported_date'].str.contains(',')][['company', 'industry', 'reported_date']]
repeated_companies = repeated_companies.drop_duplicates('company').reset_index(drop=True)

repeated_companies

,company,industry,reported_date
0,Argo AI,Transportation,"7/7/2022, 10/26/2022"
1,Blend,Fintech,"4/19/2022, 1/10/2023"
2,DataRobot,Artificial Intelligence,"8/8/2022, 8/23/2022"
3,Gemini,Crypto,"7/18/2022, 6/2/2022"
4,Homeward,"Real Estate, PropTech","8/11/2022, 11/16/2022"
5,Latch,"Smart Home, Security","5/13/2022, 8/2/2022"
6,On Deck,Business development,"5/5/2022, 8/4/2022"
7,Skillz,Esports/Video Games,"5/20/2022, 8/29/2022"
8,Socure,"cybsersecurity, predictive analytics","1/5/2023, 1/5/2023"
9,Sundae,"PropTech, Real Estate","6/28/2022, 9/13/2022"


In [105]:
# Printing the companies with two or more layoffs
print('Of the 489 lines, there are {} unique companies. {} of them have two or more layoffs registered.'.format(unique_companies, total_rows - unique_companies))
print('They are: \n{}'.format(repeated_companies))

Of the 489 lines, there are 477 unique companies. 12 of them have two or more layoffs registered.
They are: 
           company                              industry  \
0          Argo AI                        Transportation   
1            Blend                               Fintech   
2        DataRobot               Artificial Intelligence   
3           Gemini                                Crypto   
4         Homeward                 Real Estate, PropTech   
5            Latch                  Smart Home, Security   
6          On Deck                  Business development   
7           Skillz                   Esports/Video Games   
8           Socure  cybsersecurity, predictive analytics   
9           Sundae                 PropTech, Real Estate   
10  Thirty Madison                           Health Care   
11        TruePill                           Health Care   

            reported_date  
0    7/7/2022, 10/26/2022  
1    4/19/2022, 1/10/2023  
2     8/8/2022, 8/23/2022 

----

### 💡 **Analysis 1) Check which companies had more and fewer layoff**

In [ ]:
# Converting columns to their correct type
tech_layoffs['total_layoffs'] = pd.to_numeric(tech_layoffs['total_layoffs'], errors='coerce')
tech_layoffs['impacted_workforce_percentage'] = pd.to_numeric(tech_layoffs['impacted_workforce_percentage'], errors='coerce')
tech_layoffs['reported_date'] = pd.to_datetime(tech_layoffs['reported_date'], format='%d/%M/%Y')

In [9]:
tech_layoffs.sample(1)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes
74,Brex,136.0,11.0,2022-01-10 00:11:00,Fintech,San Francisco,TechCrunch,Private,NaN


In [10]:
# Step 1: calculating the total employees for each company (that has "total_layoff" and "impacted_workforce_percentage" filled)
tech_layoffs['total_employees'] = ((tech_layoffs['total_layoffs'] / 100) / (tech_layoffs['impacted_workforce_percentage'] / 100)) * 100

# Step 2: rounding the "total_employees" value
tech_layoffs['total_employees'] = tech_layoffs['total_employees'].round(0)

tech_layoffs.sample(2)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,total_employees
473,Beyond Meat,240.0,NaN,2022-01-10 00:14:00,Foodtech,"Manhattan Beach, CA",CNBC,Public,NaN,NaN
258,Halcyon Health,NaN,100.0,2022-01-04 00:15:00,Health Care,New York,Axios,Private,NaN,NaN


In [11]:
# Step 3: grouping by company name and removing rows with empty "total_layoffs" or "total_employees"
top_companies = tech_layoffs.groupby('company', as_index=False)[['total_layoffs', 'total_employees']].sum()

top_companies = top_companies[top_companies['total_layoffs'] > 0]
top_companies = top_companies[top_companies['total_employees'] > 0]

# Step 4: creating total layoff percentage (in case a company had layoff twice or more)
top_companies['total_layoff_percentage'] = ((top_companies['total_layoffs'] / top_companies['total_employees']) * 100).round(1)

top_companies.sample(3)

,company,total_layoffs,total_employees,total_layoff_percentage
36,Astronomer,76.0,380.0,20.0
366,Sema4,750.0,1923.0,39.0
142,Edgio,95.0,679.0,14.0


In [12]:
# Step 5.1: categorizing the company size
company_size_conditions = [
    top_companies['total_employees'] < 100,      # <100 Employees => Small Company
    top_companies['total_employees'] < 500,      # >=100 & <500 Employees => Mid Company
    top_companies['total_employees'] < 2000,     # >=500 & <2000 Employees => Large Company
    top_companies['total_employees'] >= 2000     # >=2000 Employees => Big Tech
]

company_size_options = [
    'Small Company',
    'Mid Company',
    'Large Company',
    'Big Tech'
]

top_companies['company_size'] = np.select(company_size_conditions, company_size_options, 'Unidentified')

top_companies.sample(3)

,company,total_layoffs,total_employees,total_layoff_percentage,company_size
308,Pacaso,100.0,333.0,30.0,Mid Company
206,IRL,25.0,100.0,25.0,Mid Company
289,Ocavu,20.0,42.0,47.6,Small Company


In [13]:
# Step 5.2: merging the "company_size" column to the main dataframe
tech_layoffs = pd.merge(top_companies[['company', 'company_size']], tech_layoffs, how='left', on='company')

tech_layoffs.sample(3)

,company,company_size,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,total_employees
156,Picsart,Large Company,90.0,8.0,2022-01-05 00:17:00,Media/entertainment,Miami,The Information,Private,NaN,1125.0
53,ConsenSys,Large Company,100.0,11.0,2023-01-01 00:10:00,"Cryptocurrency, FinTech",New York,CoinDesk,Private,NaN,909.0
190,Skillz,Large Company,70.0,10.0,2022-01-05 00:20:00,Esports/Video Games,San Francisco,TechCrunch,Public,NaN,700.0


----

In [14]:
# Step 6.1: sorting by top 10 most "total_layoffs"
top_companies.sort_values('total_layoffs', ascending=False).head(5).reset_index(drop=True)

,company,total_layoffs,total_employees,total_layoff_percentage,company_size
0,Amazon,18000.0,360000.0,5.0,Big Tech
1,Meta,11000.0,84615.0,13.0,Big Tech
2,Better.com,5000.0,10000.0,50.0,Big Tech
3,Cisco,4100.0,82000.0,5.0,Big Tech
4,Twitter,3740.0,5343.0,70.0,Big Tech


In [21]:
# Analyzing 6.1: top 10 most "total_layoffs"
tech_layoffs[tech_layoffs['company'] == 'Amazon']

,company,company_size,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,total_employees
8,Amazon,Big Tech,18000.0,5.0,2023-01-01 00:05:00,"E-commerce, SaaS",Seattle,Wall Street Journal,Public,NaN,360000.0


----

In [99]:
# Step 6.2: sorting by top 10 fewer "total_layoffs"
top_companies.sort_values('total_layoffs', ascending=True).head(5).reset_index(drop=True)

,company,total_layoffs,total_employees,total_layoff_percentage,company_size
0,Woven,5.0,33.0,15.2,Small Company
1,Sourcepoint,7.0,54.0,13.0,Small Company
2,Digital Currency Group,10.0,77.0,13.0,Small Company
3,RealSelf,11.0,220.0,5.0,Mid Company
4,Abra,12.0,240.0,5.0,Mid Company


In [98]:
# Step 6.3: sorting by top 10 most "total_layoff_percentage" (important!)
top_companies.sort_values('total_layoff_percentage', ascending=False).head(10).reset_index(drop=True)

,company,total_layoffs,total_employees,total_layoff_percentage,company_size
0,Protocol Media,60.0,60.0,100.0,Small Company
1,SummerBio,101.0,101.0,100.0,Mid Company
2,Fast,400.0,400.0,100.0,Mid Company
3,Reali,140.0,140.0,100.0,Mid Company
4,Butler Hospitality,1000.0,1000.0,100.0,Large Company
5,WanderJaunt,85.0,85.0,100.0,Small Company
6,Uniphore,76.0,100.0,76.0,Mid Company
7,Parler,60.0,80.0,75.0,Small Company
8,Twitter,3740.0,5343.0,70.0,Big Tech
9,Bizzabo,220.0,367.0,59.9,Mid Company


In [97]:
# Step 6.4: sorting by top 10 fewer "total_layoff_percentage" (important!)
top_companies.sort_values('total_layoff_percentage', ascending=True).head(10).reset_index(drop=True)

,company,total_layoffs,total_employees,total_layoff_percentage,company_size
0,F5,100.0,10000.0,1.0,Big Tech
1,Freshworks,90.0,4500.0,2.0,Big Tech
2,C2FO,20.0,1000.0,2.0,Large Company
3,Amperity,13.0,433.0,3.0,Mid Company
4,WalkMe,43.0,1433.0,3.0,Large Company
5,Addepar,20.0,667.0,3.0,Large Company
6,Outbrain,38.0,1267.0,3.0,Large Company
7,Coursera,32.0,1067.0,3.0,Large Company
8,Amdocs,700.0,23333.0,3.0,Big Tech
9,Thirty Madison,24.0,800.0,3.0,Large Company


----

contar quantas vezes a empresa fez layoff
isso é negativo pra ela se fez mais de 1 vez

## Analysis 2) Rank which industry had most layoff 

## Analysis 3)  

In [ ]:
# Exporting files to .csv to read in Power BI
tech_layoffs.to_csv('../data/tech_layoffs.csv')
top_companies.to_csv('../data/top_companies.csv')